In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
from langchain_openai import OpenAIEmbeddings

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')# 3072

In [ ]:
# pinecone 에 데이터 로딩
# import os
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore

# index_name = 'tax-index2'
# pinecone_api_key = os.getenv('PINECONE_API_KEY')
# pc = Pinecone(api_key=pinecone_api_key)

# vectorstore = PineconeVectorStore.from_existing_index(
#     index_name=index_name,
#     embedding=embedding
# )

In [5]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index5'
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=pinecone_api_key)
pinecone_index = pc.Index(index_name)

vectorstore = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [8]:
#query = "연봉 5천만원인 거주자의 종합소득세는?"
query = "연봉 5천만원 직장인의 소득세는?"

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

d:\Study\agetn_work3\myenv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [10]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)

In [11]:
ai_message = qa_chain.invoke({"query": query})

In [12]:
ai_message

{'query': '연봉 5천만원 직장인의 소득세는?',
 'result': '기본세율에 따라 근로소득 세금을 계산해야 합니다. 구체적인 세율은 연봉 및 공제 항목에 따라 달라질 수 있으며, 일반적으로 여러 공제 및 비과세 항목을 고려하여 계산됩니다. 연봉 5천만 원의 근로소득에 대한 세금은 이러한 계산을 통해 구체화되어야 합니다.'}

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt2 = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    그런 경우에는 질문만 리턴해주세요
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt2 | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

In [28]:
new_question = dictionary_chain.invoke({"question": query})

In [29]:
new_question

'연봉 5천만원 거주자의 소득세는?'

In [30]:
ai_response = tax_chain.invoke({"question": query})

In [31]:
ai_response

{'query': '연봉 5천만원 거주자의 소득세는?',
 'result': '연봉 5천만원의 경우, 소득세 계산은 다음과 같습니다: 기본적으로 1,400만원 초과 5,000만원 이하 구간에 해당됩니다. 이 경우, 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)로 계산됩니다. 따라서, 5,000만원에 대한 소득세는 84만원 + (3,600만원 × 0.15) = 84만원 + 540만원 = 624만원이 됩니다.'}